In [1]:
import geopandas as gpd
import pandas as pd
import os

# busroute.shp 的 欄位名稱
route_routename_col = 'RouteNameZ'
route_direction_col = 'Direction'

# Seq 
seq_routename_col = 'RouteName'
seq_direction_col = 'Direction'
seq_lat_col = 'Lat'
seq_lng_col = 'Lon'

busroute = gpd.read_file(os.path.join(os.getcwd(),'..', 'input','Shp','BusRoute.shp'))
seq = seq = pd.read_csv(os.path.join(os.getcwd(),'..', 'input','seq.csv'))